In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None) #To Display all columns
bankData = pd.read_csv('../input/marketing-banking-file/bank-additional-full.csv')

Attribute Information:<br>Input variables:<br># bank client data:<br>1 - age (numeric)<br>2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>6 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>7 - loan: has personal loan? (categorical: 'no','yes','unknown')<br># related with the last contact of the current campaign:<br>8 - contact: contact communication type (categorical: 'cellular','telephone')<br>9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br># other attributes:<br>12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br># social and economic context attributes<br>16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>17 - cons.price.idx: consumer price index - monthly indicator (numeric)<br>18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)<br>19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>20 - nr.employed: number of employees - quarterly indicator (numeric)<br>
<b>Output variable (desired target):<br></b>
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

### Initial Data Exploration

In [3]:
bankData.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
bankData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

- It seems there are no values which are missing in bank marketing data set.

In [5]:
bankData.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Data Cleaning

Columns emp.var.rate,cons.price.idx,cons.conf.idx,euribor.3m and nr.employed does not seem to be meaningful.Converting them to appropiate names

In [6]:
bankData.rename({'emp.var.rate':'Employment_Variation_Rate',
                'cons.price.idx': 'Consumer_Price_Index',
                 'cons.conf.idx':'Consumer_Confidence_Index',
                 'euribor3m':'Euribor_3_Month_Rate',
                 'nr.employed':'No_Employees',
                 'y':'Target_Variable'
                },axis=1,inplace=True)

In [7]:
bankData.columns


Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'Employment_Variation_Rate',
       'Consumer_Price_Index', 'Consumer_Confidence_Index',
       'Euribor_3_Month_Rate', 'No_Employees', 'Target_Variable'],
      dtype='object')

As mentioned earlier <i>Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.</i><br>

In [8]:
#Dropping duration column
bankData.drop('duration',axis=1,inplace=True)

In [9]:
#Checking if duration is dropped
bankData.shape

(41188, 20)

Month and day of the week are not required as it is not required to know what was last month,day of week when the customer was contacted

In [10]:
#Dropping month and day_of_week
bankData.drop(columns = ['month','day_of_week'],axis=1,inplace=True)

In [11]:
#Checking if month and day of week is dropped.
bankData.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'campaign', 'pdays', 'previous', 'poutcome',
       'Employment_Variation_Rate', 'Consumer_Price_Index',
       'Consumer_Confidence_Index', 'Euribor_3_Month_Rate', 'No_Employees',
       'Target_Variable'],
      dtype='object')

### Handling Missing Values

In [12]:
#Checking for missing values
bankData.isnull().sum()

age                          0
job                          0
marital                      0
education                    0
default                      0
housing                      0
loan                         0
contact                      0
campaign                     0
pdays                        0
previous                     0
poutcome                     0
Employment_Variation_Rate    0
Consumer_Price_Index         0
Consumer_Confidence_Index    0
Euribor_3_Month_Rate         0
No_Employees                 0
Target_Variable              0
dtype: int64

There dosen't seem to be any values missing but need to check if there are any unkown's assigned to any columns

In [13]:
#Checking distribution for age
bankData.age.describe()

count    41188.00000
mean        40.02406
std         10.42125
min         17.00000
25%         32.00000
50%         38.00000
75%         47.00000
max         98.00000
Name: age, dtype: float64

Age looks fine and cannot see any values like -5 or 0 which are not standard.

In [14]:
#Checking distribution of values for job
bankData.job.value_counts()/bankData.shape[0] * 100

admin.           25.303486
blue-collar      22.467709
technician       16.371273
services          9.636302
management        7.099155
retired           4.175974
entrepreneur      3.535010
self-employed     3.450034
housemaid         2.573565
unemployed        2.461882
student           2.124405
unknown           0.801204
Name: job, dtype: float64

There are around 0.8% of values unkown for job.At first look it looks like they can be dropped.

In [15]:
#Checking further where job is unkown.
bankData[bankData['job'] == 'unknown']

,age,job,marital,education,default,housing,loan,contact,campaign,pdays,previous,poutcome,Employment_Variation_Rate,Consumer_Price_Index,Consumer_Confidence_Index,Euribor_3_Month_Rate,No_Employees,Target_Variable
29,55,unknown,married,university.degree,unknown,unknown,unknown,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
35,55,unknown,married,basic.4y,unknown,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
73,57,unknown,married,unknown,unknown,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
91,57,unknown,married,unknown,unknown,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
144,38,unknown,divorced,high.school,unknown,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40370,59,unknown,married,unknown,no,no,no,cellular,1,6,1,success,-1.7,94.027,-38.3,0.900,4991.6,yes
40428,64,unknown,married,unknown,no,yes,no,telephone,4,999,0,nonexistent,-1.7,94.027,-38.3,0.905,4991.6,yes
40656,67,unknown,divorced,unknown,unknown,yes,no,cellular,2,6,2,success,-1.1,94.199,-37.5,0.880,4963.6,yes
41005,63,unknown,married,professional.course,no,no,no,cellular,1,6,1,success,-1.1,94.601,-49.5,1.025,4963.6,no


In [16]:
bankData[bankData['job'] == 'unknown']['pdays'].value_counts()

999    310
3        8
6        6
2        2
1        1
8        1
4        1
13       1
Name: pdays, dtype: int64

It looks like for most of the values in job where it is unkown the client has not been contacted earlier which is indicated by 999 value.So, the target variable in this case will not mean anything and it is better to drop the unkown values in job.

In [17]:
bankData[bankData['job'] != 'unknown']

,age,job,marital,education,default,housing,loan,contact,campaign,pdays,previous,poutcome,Employment_Variation_Rate,Consumer_Price_Index,Consumer_Confidence_Index,Euribor_3_Month_Rate,No_Employees,Target_Variable
0,56,housemaid,married,basic.4y,no,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [18]:
print(f'Before dropping unknown for job:{bankData.shape}')
unknown = len(bankData[bankData['job'] == 'unknown'])
print(f'Count of unknown for job:{unknown}')
bankData= bankData[bankData['job'] != 'unknown']
print(f'After dropping unknown for job:{bankData.shape}')


Before dropping unknown for job:(41188, 18)
Count of unknown for job:330
After dropping unknown for job:(40858, 18)


In [19]:
#Checking distribution of values for marital
bankData.marital.value_counts()/bankData.shape[0] * 100

married     60.438592
single      28.131578
divorced    11.256058
unknown      0.173773
Name: marital, dtype: float64

In [20]:
#Checking further where job is unkown.
bankData[bankData['marital'] == 'unknown']

,age,job,marital,education,default,housing,loan,contact,campaign,pdays,previous,poutcome,Employment_Variation_Rate,Consumer_Price_Index,Consumer_Confidence_Index,Euribor_3_Month_Rate,No_Employees,Target_Variable
40,58,management,unknown,university.degree,no,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
390,59,retired,unknown,university.degree,unknown,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
413,57,retired,unknown,basic.4y,no,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1493,33,services,unknown,high.school,no,yes,yes,telephone,2,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
3167,57,retired,unknown,basic.4y,no,yes,no,telephone,10,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38550,31,entrepreneur,unknown,university.degree,no,no,no,cellular,1,999,0,nonexistent,-3.4,92.431,-26.9,0.722,5017.5,yes
38579,31,entrepreneur,unknown,university.degree,no,yes,no,telephone,4,999,1,failure,-3.4,92.431,-26.9,0.722,5017.5,no
39322,30,technician,unknown,university.degree,no,no,no,cellular,2,11,2,success,-1.8,93.369,-34.8,0.639,5008.7,yes
39323,30,technician,unknown,university.degree,no,no,no,cellular,2,6,2,success,-1.8,93.369,-34.8,0.639,5008.7,yes


In [21]:
bankData[bankData['marital'] == 'unknown']['pdays'].value_counts()

999    66
6       2
4       1
11      1
16      1
Name: pdays, dtype: int64




It looks like for most of the values in marital where it is unkown the client has not been contacted earlier which is indicated by 999 value.So, the target variable in this case will not mean anything and it is better to drop the unkown values in marital.

In [22]:
print(f'Before dropping unknown for marital:{bankData.shape}')
unknown = len(bankData[bankData['marital'] == 'unknown'])
print(f'Count of unknown for marital:{unknown}')
bankData= bankData[bankData['marital'] != 'unknown']
print(f'After dropping unknown for marital:{bankData.shape}')


Before dropping unknown for marital:(40858, 18)
Count of unknown for marital:71
After dropping unknown for marital:(40787, 18)


In [23]:
#Checking distribution of values for education
bankData.education.value_counts()/bankData.shape[0] * 100

university.degree      29.656508
high.school            23.203472
basic.9y               14.725280
professional.course    12.810454
basic.4y               10.096354
basic.6y                5.550788
unknown                 3.913011
illiterate              0.044132
Name: education, dtype: float64

In [24]:
#Checking further where education is unkown.
bankData[bankData['education'] == 'unknown']

,age,job,marital,education,default,housing,loan,contact,campaign,pdays,previous,poutcome,Employment_Variation_Rate,Consumer_Price_Index,Consumer_Confidence_Index,Euribor_3_Month_Rate,No_Employees,Target_Variable
7,41,blue-collar,married,unknown,unknown,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
10,41,blue-collar,married,unknown,unknown,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
26,59,technician,married,unknown,no,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
30,46,admin.,married,unknown,no,no,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
31,59,technician,married,unknown,no,yes,no,telephone,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41118,34,technician,married,unknown,no,yes,no,cellular,2,999,2,failure,-1.1,94.767,-50.8,1.046,4963.6,no
41120,60,admin.,married,unknown,no,no,no,cellular,2,999,0,nonexistent,-1.1,94.767,-50.8,1.046,4963.6,no
41122,34,technician,married,unknown,no,no,no,cellular,3,999,0,nonexistent,-1.1,94.767,-50.8,1.046,4963.6,yes
41135,54,technician,married,unknown,no,yes,no,cellular,1,999,1,failure,-1.1,94.767,-50.8,1.041,4963.6,no


In [25]:
bankData[bankData['education'] == 'unknown']['pdays'].value_counts()

999    1505
3        28
6        18
4         7
9         7
7         6
10        4
12        3
17        2
15        2
14        2
18        2
13        2
16        2
20        1
5         1
8         1
11        1
2         1
1         1
Name: pdays, dtype: int64

It looks like for most of the values in education where it is unkown the client has not been contacted earlier which is indicated by 999 value.So, the target variable in this case will not mean anything and it is better to drop the unkown values in education as percentage is very less(3%).

In [26]:
print(f'Before dropping unknown for education:{bankData.shape}')
unknown = len(bankData[bankData['education'] == 'unknown'])
print(f'Count of unknown for education:{unknown}')
bankData= bankData[bankData['education'] != 'unknown']
print(f'After dropping unknown for education:{bankData.shape}')


Before dropping unknown for education:(40787, 18)
Count of unknown for education:1596
After dropping unknown for education:(39191, 18)


In [27]:

#Checking distribution where default is unkown.
bankData.default.value_counts()/bankData.shape[0] * 100

no         79.671353
unknown    20.320992
yes         0.007655
Name: default, dtype: float64

20% of values are missing and cannot be neglected .This is a categorical column and the unkown values need to be handled

In [28]:
bankData[bankData['default'] == 'unknown']['pdays'].value_counts()

999    7907
6        17
3        12
4         4
2         4
10        4
12        3
1         3
5         2
7         2
9         2
0         1
8         1
11        1
14        1
Name: pdays, dtype: int64

It looks like for most of the values in default where it is unkown the client has not been contacted earlier which is indicated by 999 value.So, the target variable in this case will not mean anything and it is better to drop the unkown values where pdays= 999  and for other unkown values replace with mode.

In [29]:
print(f'Before dropping unknown for default:{bankData.shape}')
unknown = len(bankData[(bankData['default'] == 'unknown') & 
              (bankData['pdays'] == 999)
                       ])
print(f'Count of unknown for default:{unknown}')
bankData= bankData[~((bankData['default'] == 'unknown') & 
              (bankData['pdays'] == 999))]
print(f'After dropping unknown for default:{bankData.shape}')


Before dropping unknown for default:(39191, 18)
Count of unknown for default:7907
After dropping unknown for default:(31284, 18)


In [30]:
#Replacing the left unkown values in default with mode.
default_mode = bankData.default.mode()[0]
bankData['default'] = bankData['default'].apply(lambda x: default_mode if x=='unknown' else x)

In [31]:
#Checking distribution where housing is unkown.
bankData.default.value_counts()/bankData.shape[0] * 100

no     99.99041
yes     0.00959
Name: default, dtype: float64

In [32]:
#Checking distribution where housing is unkown.
bankData.housing.value_counts()/bankData.shape[0] * 100

yes        52.921621
no         44.712952
unknown     2.365426
Name: housing, dtype: float64

In [33]:
bankData[bankData['housing'] == 'unknown']['pdays'].value_counts()

999    707
3       10
6        7
10       4
4        3
8        2
7        2
9        2
12       2
2        1
Name: pdays, dtype: int64

It looks like for most of the values in housing where it is unkown the client has not been contacted earlier which is indicated by 999 value.So, the target variable in this case will not mean anything and it is better to drop the unkown values in education as percentage is very less(2%).

In [34]:
print(f'Before dropping unknown for housing:{bankData.shape}')
unknown = len(bankData[bankData['housing'] == 'unknown'])
print(f'Count of unknown for housing:{unknown}')
bankData= bankData[bankData['housing'] != 'unknown']
print(f'After dropping unknown for housing:{bankData.shape}')


Before dropping unknown for housing:(31284, 18)
Count of unknown for housing:740
After dropping unknown for housing:(30544, 18)


In [35]:
#Checking distribution for loan
bankData.loan.value_counts()/bankData.shape[0] * 100

no     84.356993
yes    15.643007
Name: loan, dtype: float64

No unkown values

In [36]:
#Checking distribution for contact
bankData.contact.value_counts()/bankData.shape[0] * 100

cellular     67.103195
telephone    32.896805
Name: contact, dtype: float64

No unkown values


In [37]:
#Checking distribution for campaign
bankData.campaign.describe()

count    30544.000000
mean         2.520233
std          2.718412
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         43.000000
Name: campaign, dtype: float64

No values are there which look out of place.

In [38]:
#Checking distribution for pdays
bankData.pdays.describe()

count    30544.000000
mean       954.589085
std        205.259166
min          0.000000
25%        999.000000
50%        999.000000
75%        999.000000
max        999.000000
Name: pdays, dtype: float64

No values are there which look out of place.
But the variance is not much in pdays as most of the values are 999 indicating client has not been contacted.

In [39]:
#Removing pdays column
bankData.drop('pdays',axis=1,inplace=True)

In [40]:
#Checking distribution for poutcome
bankData.poutcome.value_counts()/bankData.shape[0] * 100

nonexistent    84.586171
failure        11.344290
success         4.069539
Name: poutcome, dtype: float64

In [41]:
bankData.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'campaign', 'previous', 'poutcome',
       'Employment_Variation_Rate', 'Consumer_Price_Index',
       'Consumer_Confidence_Index', 'Euribor_3_Month_Rate', 'No_Employees',
       'Target_Variable'],
      dtype='object')

In [42]:
#Checking distribution for contact
bankData.Target_Variable.value_counts()/bankData.shape[0] * 100

no     87.280644
yes    12.719356
Name: Target_Variable, dtype: float64

- No values other then yes/no are present.
- Looks like an imbalanced dataset as most of the values are no .